In [1]:
import psycopg2
import torch
import pickle
import sqlite3
import pandas as pd
import numpy as np

Tables

```sql
CREATE TABLE `djmdRecommendLike` (
  `ID` VARCHAR(255) PRIMARY KEY,  -- random uuid
  `ContentID1` VARCHAR(255) DEFAULT NULL,  -- song id 1
  `ContentID2` VARCHAR(255) DEFAULT NULL,  -- song id 2
  `LikeRate` INTEGER DEFAULT NULL,  -- NULL
  `DataCreatedH` INTEGER DEFAULT NULL, -- I think these fields are uint64 ms timestamp in 2 ints
  `DataCreatedL` INTEGER DEFAULT NULL, 
  `UUID` VARCHAR(255) DEFAULT NULL,  -- random uuid (a differnt one)
  `rb_data_status` INTEGER DEFAULT 0,  -- all 0s
  `rb_local_data_status` INTEGER DEFAULT 0, 
  `rb_local_deleted` TINYINT(1) DEFAULT 0, 
  `rb_local_synced` TINYINT(1) DEFAULT 0, 
  `usn` BIGINT DEFAULT NULL,  -- null
  `rb_local_usn` BIGINT DEFAULT NULL,  -- localUpdateCount value
  `created_at` DATETIME NOT NULL, 
  `updated_at` DATETIME NOT NULL
);
-- INSERT INTO agentRegistry VALUES('localUpdateCount',NULL,NULL,692204,NULL,NULL,NULL,NULL,NULL,NULL,NULL,'2022-05-20 05:28:00.478 +00:00','2022-05-20 05:28:00.478 +00:00');
CREATE TABLE `agentRegistry` (
  `registry_id` VARCHAR(255) PRIMARY KEY, 
  `id_1` VARCHAR(255) DEFAULT NULL, 
  `id_2` VARCHAR(255) DEFAULT NULL, 
  `int_1` BIGINT DEFAULT NULL, 
  `int_2` BIGINT DEFAULT NULL, 
  `str_1` VARCHAR(255) DEFAULT NULL, 
  `str_2` VARCHAR(255) DEFAULT NULL, 
  `date_1` DATETIME DEFAULT NULL, 
  `date_2` DATETIME DEFAULT NULL, 
  `text_1` TEXT DEFAULT NULL, 
  `text_2` TEXT DEFAULT NULL, 
  `created_at` DATETIME NOT NULL, 
  `updated_at` DATETIME NOT NULL
);
```

In [3]:
def connect():
    return psycopg2.connect(database="music_embeddings",
                        host="localhost",
                        user="maxj",
                        password="",
                        port="5432")


In [4]:
def get(path):
    conn = connect()
    c = conn.cursor()
    c.execute('select id, bpm, embedding_musicgen_medium from tracks where path=%s limit 1', (path,))
    id, bpm, emb = c.fetchone()
    c.close()
    conn.close()
    return id, bpm, emb


In [5]:
def sim(id, bpm, emb, limit=5):
    conn = connect()
    c = conn.cursor()
    c.execute(f'''
select 
  id, 
  path, 
  artist, 
  title, 
  key,
  bpm / 100,
  (1 - (embedding_musicgen_medium <=> %s)) as cosine_sim 
from 
  tracks 
where 
  id != %s 
  and embedding_musicgen_medium is not null 
  and (
    bpm >= 0.92 * %s and bpm <= 1.08 * %s 
    or bpm >= 1.84 * %s and bpm <= 2.16 * %s 
    or bpm >= 0.46 * %s and bpm <= 0.54 * %s
  )
order by 
  cosine_sim desc 
limit 
  {limit}''', (emb, id, bpm, bpm, bpm, bpm, bpm, bpm))
    # c.execute('select id, path, artist, title from tracks where id != %s order by embedding_musicgen_medium <=> %s limit 5', (id, emb))
    rows = c.fetchall()
    c.close()
    conn.close()
    return pd.DataFrame.from_records(rows, columns=["id", "path", "artist", "title", "key", "bpm", "cosine_sim"])

In [6]:
def get_paths():
    conn = connect()
    c = conn.cursor()
    c.execute('select path from tracks where embedding_musicgen_medium is not null and bpm > 0')
    paths = c.fetchall()
    c.close()
    conn.close()
    return paths

paths = get_paths()

In [7]:
relateds = set() # (id1, id2)

for path in paths:
    id, bpm, emb = get(path)
    ids = sim(id, bpm, emb, 6)['id']
    relateds.update((tuple(sorted([id, i])) for i in ids))

In [8]:
import uuid

In [9]:
relateds = sorted(relateds)

lines = [
    "PRAGMA KEY='402fd482c38817c35ffa8ffb8c7d93143b749e7d315df7a81732a1ff43608497';",
    "DELETE FROM djmdRecommendLike;"
]
for (id1, id2) in relateds:
    lines += [f"INSERT INTO djmdRecommendLike (ID, ContentID1, ContentID2, created_at, updated_at) VALUES ('{uuid.uuid4()}', {id1}, {id2}, '2023-10-24 04:30:03.956 +00:00', '2023-10-24 04:30:03.956 +00:00');"]
with open('pop.sql', 'w') as f:
    for l in lines:
        f.write(l + '\n')

In [10]:
import subprocess

def is_app_running(app_name):
    try:
        # Run the 'pgrep' command to search for the running process
        result = subprocess.run(['pgrep', app_name], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # If the return code is 0, the app is running
        return result.returncode == 0
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

# Example usage
if is_app_running('rekordbox'):
    raise ValueError("close rekordbox")

res = subprocess.run(['sqlcipher', '-init', 'pop.sql', '/Users/maxj/Library/Pioneer/rekordbox/master.db', '.quit'], check=True)
res

ok


CompletedProcess(args=['sqlcipher', '-init', 'pop.sql', '/Users/maxj/Library/Pioneer/rekordbox/master.db', '.quit'], returncode=0)